In [1]:
import kbible
import pandas as pd

In [2]:
b = kbible.read_full_bible("개역개정판성경")

## find verse id or matched verse

In [4]:
def find_id(bible, book=[], chapter=[], verse=[]):
    """ find index on full bible database """

    # check books
    books = set(bible['book'])
    books_long = set(bible['book_long'])
    
    if len(book) == 0:
        book = books[0]
    if isinstance(book, str):
        book = [book]
        
    print('... search book:{}'.format(book))
    result = bible.loc[bible.book.isin(book) | bible.book_long.isin(book)]
        
    # check chapter
    if isinstance(chapter, int):
        chapter = [chapter]
    if len(chapter) == 0:
        return result
    
    print('... search chapter: {}'.format(chapter))
    result = result.loc[bible.chapter.isin(chapter)]
        
    # check verse
    if isinstance(verse, int):
        verse = [verse]
    if len(verse) == 0:
        return result
    
    print('... search verse: {}'.format(verse))
    result = result.loc[bible.verse.isin(verse)]
        
    if len(result) > 0:
        return result
    else:
        print("... not found: {}, {}, {}".format(book, chapter, verse))

In [5]:
find_id(b, book=["창","출"], chapter=[1,2], verse=[1])

... search book:['창', '출']
... search chapter: [1, 2]
... search verse: [1]


,book,chapter,verse,text,order,book_long,eng,on
0,창,1,1,태초에 하나님이 천지를 창조하시니라,1,창세기,Genesis,구약
31,창,2,1,천지와 만물이 다 이루어지니라,1,창세기,Genesis,구약
1533,출,1,1,야곱과 함께 각각 자기 가족을 데리고 애굽에 이른 이스라엘 아들들의 이름은 이러하니,2,출애굽기,Exodus,구약
1555,출,2,1,레위 가족 중 한 사람이 가서 레위 여자에게 장가 들어,2,출애굽기,Exodus,구약


In [13]:
b.loc[[0, 31, 1533, 1555]]

,book,chapter,verse,text,order,book_long,eng,on
0,창,1,1,태초에 하나님이 천지를 창조하시니라,1,창세기,Genesis,구약
31,창,2,1,천지와 만물이 다 이루어지니라,1,창세기,Genesis,구약
1533,출,1,1,야곱과 함께 각각 자기 가족을 데리고 애굽에 이른 이스라엘 아들들의 이름은 이러하니,2,출애굽기,Exodus,구약
1555,출,2,1,레위 가족 중 한 사람이 가서 레위 여자에게 장가 들어,2,출애굽기,Exodus,구약


## extract verse by short search string

In [15]:
set(b.book)

{'갈',
 '겔',
 '계',
 '고전',
 '고후',
 '골',
 '나',
 '눅',
 '느',
 '단',
 '대상',
 '대하',
 '딛',
 '딤전',
 '딤후',
 '레',
 '렘',
 '롬',
 '룻',
 '마',
 '막',
 '말',
 '몬',
 '미',
 '민',
 '벧전',
 '벧후',
 '빌',
 '사',
 '살전',
 '살후',
 '삼상',
 '삼하',
 '삿',
 '수',
 '스',
 '슥',
 '습',
 '시',
 '신',
 '아',
 '암',
 '애',
 '약',
 '에',
 '엡',
 '옵',
 '왕상',
 '왕하',
 '요',
 '요삼',
 '요이',
 '요일',
 '욘',
 '욜',
 '욥',
 '유',
 '잠',
 '전',
 '창',
 '출',
 '학',
 '합',
 '행',
 '호',
 '히'}

In [57]:
def extract_bystr(bible, sstr, form="pd"):
    """ extract verse by short search string 
    sstr: "창3:16", "고후5:3", '요일1:1'
    - no space
    - one separator
    """
    
    # remove all spaces
    sstr = sstr.replace(" ", "")
    
    # find components
    verses = sstr.split(':')[1]
    head = sstr.split(':')[0]
    
    book = ''.join(filter(str.isalpha, head))
    chapter = ''.join(filter(str.isdigit, head))
    chapter = int(chapter)
    
    # check , in verse
    if verses.find(',') > 0:
        verses = verses.split(',')
    # check - in verse
    elif verses.find('-') > 0:
        start = verses.split('-')[0]
        end = verses.split('-')[1]
        try:
            verses = list(range(int(start), int(end)+1))
        except:
            print('... wrong format: {}'.format(sstr))
            return 0
    verses = [int(v) for v in verses]
    
    #print(book, chapter, verses)

    # return verses
    res = find_id(bible, book=book, chapter=chapter, verse=verses)
    if len(res) == 0:
        print('... no result!')
        return []
    
    if form == "pd":
        return res
    if form == "string":
        return '. '.join(res['text'].values)
    if form == "md":
        msg = "`{}` ".format(sstr)
        return msg + '. '.join(res['text'].values)
        

In [55]:
extract_bystr(b, "고후5:3,4,5")

... search book:['고후']
... search chapter: [5]
... search verse: [3, 4, 5]


,book,chapter,verse,text,order,book_long,eng,on
28873,고후,5,3,이렇게 입음은 우리가 벗은 자들로 발견되지 않으려 함이라,8,고린도후서,2_Corinthians,신약
28874,고후,5,4,참으로 이 장막에 있는 우리가 짐 진 것 같이 탄식하는 것은 벗고자 함이 아니요 오...,8,고린도후서,2_Corinthians,신약
28875,고후,5,5,곧 이것을 우리에게 이루게 하시고 보증으로 성령을 우리에게 주신 이는 하나님이시니라,8,고린도후서,2_Corinthians,신약


In [49]:
print(extract_bystr(b, "고후5:3-5", form="string"))

... search book:['고후']
... search chapter: [5]
... search verse: [3, 4, 5]
이렇게 입음은 우리가 벗은 자들로 발견되지 않으려 함이라. 참으로 이 장막에 있는 우리가 짐 진 것 같이 탄식하는 것은 벗고자 함이 아니요 오히려 덧입고자 함이니 죽을 것이 생명에 삼킨 바 되게 하려 함이라. 곧 이것을 우리에게 이루게 하시고 보증으로 성령을 우리에게 주신 이는 하나님이시니라


In [51]:
print(extract_bystr(b, "고후5:3-5", form="md"))

... search book:['고후']
... search chapter: [5]
... search verse: [3, 4, 5]
`고후5:3-5` 이렇게 입음은 우리가 벗은 자들로 발견되지 않으려 함이라. 참으로 이 장막에 있는 우리가 짐 진 것 같이 탄식하는 것은 벗고자 함이 아니요 오히려 덧입고자 함이니 죽을 것이 생명에 삼킨 바 되게 하려 함이라. 곧 이것을 우리에게 이루게 하시고 보증으로 성령을 우리에게 주신 이는 하나님이시니라


In [56]:
print(extract_bystr(b, "고후5:3", form="md"))

... search book:['고후']
... search chapter: [5]
... search verse: [3]
`고후5:3` 이렇게 입음은 우리가 벗은 자들로 발견되지 않으려 함이라
